In previous projects we have seen and analyzed the Neighborhoods in cities such as Toronto and New York. This time, and as I live in Chicago, the main gal is to determine which is the best neighborhood, in terms of restaurants, in Chicago. In addition, we will group the by clusters to see which of them are similar, these means, whicho other neighborhoods are also good to eat.

The dataset I will used is the next one: https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago.

In [1]:
import numpy as np # library to handle data in a vectorized manner


import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'    
wikitables =pd.read_html(page, index_col=0, attrs={"class":"wikitable"})     
wikitables[0].to_csv('chicago.csv') #Just the first table is important

In [3]:
df=pd.read_csv('canada.csv', header=1) 

In [4]:
df.head()

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [5]:
df['Borough']=df['Community area']

In [6]:
del df['Community area']


In [7]:
df=df[df.Borough.str.contains(',')!=True].reset_index()
# We sort the values to get all the same Boroughs together
df.sort_values('Borough', inplace=True)
df=df.reset_index()
del df['level_0']
del df['index']

df.head()

,Neighborhood,Borough
0,Albany Park,Albany Park
1,Mayfair,Albany Park
2,North Mayfair,Albany Park
3,Ravenswood Manor,Albany Park
4,Archer Heights,Archer Heights


In [8]:
df.sort_values('Borough')# We sort the values to get all the same Postalcodes together


b_aux=''
nb_aux=''
count=1
#We use a for to check the df and get the index
for i in df.index:
    #We store de Postcode and neighbourhood of the row i
    b=df['Borough'][i]
    nb=df['Neighborhood'][i]
    #We see if the postal code is the same as the previous one, if it is we modify the value and drop the duplicates 
    if b==b_aux:
        df['Neighborhood'][i-count]=nb_aux+','+' '+nb
        df.drop(i, axis=0, inplace=True) 
        nb=nb_aux+','+' '+nb
        count = count+1
    else:
        count=1
    #update auxiliari variables
    b_aux=b
    nb_aux=nb
    
df=df.reset_index()
del df['index']
df.head()

,Neighborhood,Borough
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Albany Park
1,Archer Heights,Archer Heights
2,"Armour Square, Chinatown, Wentworth Gardens",Armour Square
3,"Wrightwood, Scottsdale, Crestline, Beverly Vie...",Ashburn
4,"Auburn Gresham, Gresham",Auburn Gresham


In [9]:
df.shape

(78, 2)

In [10]:
address = 'Chicago'

geolocator = Nominatim(user_agent="ch_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


In [11]:
import time
df['Latitude']=''
df['Longitude']=''
for i in df.index:
    address=df['Borough'][i]+',' +' Chicago'
    geolocator = Nominatim(user_agent="borough_explorer")
    location = geolocator.geocode(address)
    df['Latitude'][i] = location.latitude
    df['Longitude'][i] = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(df['Borough'][i], df['Latitude'][i], df['Longitude'][i]))
    time.sleep(1)

The geograpical coordinate of Albany Park are 41.9719367, -87.7161739.
The geograpical coordinate of Archer Heights are 41.8114215, -87.7261651.
The geograpical coordinate of Armour Square are 41.8400333, -87.633107.
The geograpical coordinate of Ashburn are 41.7475332, -87.7111628.
The geograpical coordinate of Auburn Gresham are 41.750474, -87.6643042.
The geograpical coordinate of Austin are 41.8878759, -87.7648508.
The geograpical coordinate of Avalon Park are 41.7450346, -87.5886584.
The geograpical coordinate of Avondale are 41.9389208, -87.711168.
The geograpical coordinate of Belmont Cragin are 41.9316983, -87.7686699.
The geograpical coordinate of Beverly are 41.7181532, -87.6717674.
The geograpical coordinate of Bridgeport are 41.8379385, -87.651028.
The geograpical coordinate of Brighton Park are 41.8189217, -87.6989421.
The geograpical coordinate of Burnside are 41.7300345, -87.596714.
The geograpical coordinate of Calumet Heights are 41.7300347, -87.5792134.
The geograpica

In [76]:
# create map of Toronto using latitude and longitude values
map_Chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Chicago)  
    
map_Chicago

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
CLIENT_ID = 'ABOH3HILAFKQERTVMABZRXWSHAGAMITKCZ2W1BM3CIG1LSWV' # your Foursquare ID
CLIENT_SECRET = 'VXRXHMONNKW1X53B2CIGLG5AID0Z5DRCUCLKF4Y5A22MNTZ4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
query='restaurant'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']  
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: ABOH3HILAFKQERTVMABZRXWSHAGAMITKCZ2W1BM3CIG1LSWV
CLIENT_SECRET:VXRXHMONNKW1X53B2CIGLG5AID0Z5DRCUCLKF4Y5A22MNTZ4


In [16]:
Chicago_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Albany Park, Mayfair, North Mayfair, Ravenswood Manor
Archer Heights
Armour Square, Chinatown, Wentworth Gardens
Wrightwood, Scottsdale, Crestline, Beverly View, Parkview, Ashburn, Ashburn Estates
Auburn Gresham, Gresham
The Island, South Austin, Galewood, North Austin
Avalon Park, Marynook, Stony Island Park
Avondale, Wacławowo, Jackowo
Cragin, Brickyard, Hanson Park, Belmont Central
East Beverly, Beverly, West Beverly
Bridgeport
Brighton Park
Burnside
Pill Hill, Calumet Heights
East Chatham, West Chatham, Chatham
Chicago Lawn, Marquette Park, Lithuanian Plaza
Clearing West, Chrysler Village, Clearing East
The Gap, Lake Meadows, Stateway Gardens, Groveland Park, Dearborn Homes, South Commons, Bronzeville, Prairie Shores
Schorsch Village, Irving Woods, Belmont Terrace, Belmont Heights, Dunning
East Garfield Park, Fifth City
East Side
Lakewood / Balmoral, Edgewater Glen, Andersonville, Edgewater, Edgewater Beach
Edison Park
Englewood, Hamilton Park
South Edgebrook, Edgebrook, Forest Gle

In [17]:
# one hot encoding
Chicago_onehot = pd.get_dummies(Chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Chicago_onehot['Neighborhood'] = Chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Chicago_onehot.columns[-1]] + list(Chicago_onehot.columns[:-1])
Toronto_onehot = Chicago_onehot[fixed_columns]

Chicago_onehot.head()

,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Albany Park, Mayfair, North Mayfair, Ravenswoo..."


In [18]:
Chicago_grouped = Chicago_onehot.groupby('Neighborhood').mean().reset_index()
Chicago_grouped

,Neighborhood,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Noodle House,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.307692,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Archer Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667
2,"Armour Square, Chinatown, Wentworth Gardens",0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.125000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,"Auburn Gresham, Gresham",0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [57]:
num_top_venues = 5

for hood in Chicago_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Chicago_grouped[Chicago_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park, Mayfair, North Mayfair, Ravenswood Manor----
                venue  freq
0  Mexican Restaurant  0.31
1      Sandwich Place  0.15
2         Pizza Place  0.08
3               Diner  0.08
4   Korean Restaurant  0.08


----Archer Heights----
                venue  freq
0  Mexican Restaurant  0.40
1         Wings Joint  0.07
2  Chinese Restaurant  0.07
3  Italian Restaurant  0.07
4         Pizza Place  0.07


----Armour Square, Chinatown, Wentworth Gardens----
                venue  freq
0  Chinese Restaurant  0.25
1    Asian Restaurant  0.12
2              Bakery  0.12
3                Café  0.06
4      Breakfast Spot  0.06


----Auburn Gresham, Gresham----
                  venue  freq
0  Fast Food Restaurant  0.33
1      Greek Restaurant  0.17
2   American Restaurant  0.17
3             BBQ Joint  0.17
4    Dim Sum Restaurant  0.17


----Avalon Park, Marynook, Stony Island Park----
                       venue  freq
0       Fast Food Restaurant  0.17
1               Burg

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Chicago_grouped['Neighborhood']

for ind in np.arange(Chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Mexican Restaurant,Sandwich Place,Korean Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Diner,Donut Shop,Deli / Bodega
1,Archer Heights,Mexican Restaurant,Wings Joint,Seafood Restaurant,Hot Dog Joint,Italian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Restaurant,Bakery
2,"Armour Square, Chinatown, Wentworth Gardens",Chinese Restaurant,Asian Restaurant,Bakery,Italian Restaurant,American Restaurant,Hot Dog Joint,Indian Restaurant,Breakfast Spot,Sandwich Place,Café
3,"Auburn Gresham, Gresham",Fast Food Restaurant,American Restaurant,Greek Restaurant,BBQ Joint,Dim Sum Restaurant,Wings Joint,Falafel Restaurant,Deli / Bodega,Diner,Donut Shop
4,"Avalon Park, Marynook, Stony Island Park",Food,Fast Food Restaurant,Burger Joint,Breakfast Spot,Pizza Place,Sandwich Place,Fish & Chips Shop,Diner,Cajun / Creole Restaurant,Donut Shop


In [60]:
# set number of clusters
kclusters = 6

Chicago_grouped_clustering = Chicago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 2, 0, 0, 0, 0, 0, 4], dtype=int32)

In [61]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Chicago_merged = df
Chicago_merged.head()

,Neighborhood,Borough,Latitude,Longitude
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Albany Park,41.9719,-87.7162
1,Archer Heights,Archer Heights,41.8114,-87.7262
2,"Armour Square, Chinatown, Wentworth Gardens",Armour Square,41.84,-87.6331
3,"Wrightwood, Scottsdale, Crestline, Beverly Vie...",Ashburn,41.7475,-87.7112
4,"Auburn Gresham, Gresham",Auburn Gresham,41.7505,-87.6643


In [62]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Mexican Restaurant,Sandwich Place,Korean Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Diner,Donut Shop,Deli / Bodega
1,4,Archer Heights,Mexican Restaurant,Wings Joint,Seafood Restaurant,Hot Dog Joint,Italian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Restaurant,Bakery
2,0,"Armour Square, Chinatown, Wentworth Gardens",Chinese Restaurant,Asian Restaurant,Bakery,Italian Restaurant,American Restaurant,Hot Dog Joint,Indian Restaurant,Breakfast Spot,Sandwich Place,Café
3,2,"Auburn Gresham, Gresham",Fast Food Restaurant,American Restaurant,Greek Restaurant,BBQ Joint,Dim Sum Restaurant,Wings Joint,Falafel Restaurant,Deli / Bodega,Diner,Donut Shop
4,0,"Avalon Park, Marynook, Stony Island Park",Food,Fast Food Restaurant,Burger Joint,Breakfast Spot,Pizza Place,Sandwich Place,Fish & Chips Shop,Diner,Cajun / Creole Restaurant,Donut Shop


In [63]:
Chicago_merged = Chicago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Chicago_merged.head()
Chicago_merged.shape


(78, 15)

In [65]:
Chicago_merged = Chicago_merged[~Chicago_merged['Cluster Labels'].isnull()]#We eliminate non numeric values

Chicago_merged['Cluster Labels']=Chicago_merged['Cluster Labels'].astype(int)
Chicago_merged.head()

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Albany Park,41.9719,-87.7162,4,Mexican Restaurant,Sandwich Place,Korean Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Diner,Donut Shop,Deli / Bodega
1,Archer Heights,Archer Heights,41.8114,-87.7262,4,Mexican Restaurant,Wings Joint,Seafood Restaurant,Hot Dog Joint,Italian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Restaurant,Bakery
2,"Armour Square, Chinatown, Wentworth Gardens",Armour Square,41.84,-87.6331,0,Chinese Restaurant,Asian Restaurant,Bakery,Italian Restaurant,American Restaurant,Hot Dog Joint,Indian Restaurant,Breakfast Spot,Sandwich Place,Café
3,"Wrightwood, Scottsdale, Crestline, Beverly Vie...",Ashburn,41.7475,-87.7112,3,Food,BBQ Joint,Pizza Place,Italian Restaurant,Wings Joint,Eastern European Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
4,"Auburn Gresham, Gresham",Auburn Gresham,41.7505,-87.6643,2,Fast Food Restaurant,American Restaurant,Greek Restaurant,BBQ Joint,Dim Sum Restaurant,Wings Joint,Falafel Restaurant,Deli / Bodega,Diner,Donut Shop


In [66]:
Chicago_merged.shape

(70, 15)

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Chicago_merged['Latitude'], Chicago_merged['Longitude'], Chicago_merged['Neighborhood'], Chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 0, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Armour Square,Chinese Restaurant,Asian Restaurant,Bakery,Italian Restaurant,American Restaurant,Hot Dog Joint,Indian Restaurant,Breakfast Spot,Sandwich Place,Café
5,Austin,Wings Joint,Italian Restaurant,Japanese Restaurant,Donut Shop,Ethiopian Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Eastern European Restaurant
6,Avalon Park,Food,Fast Food Restaurant,Burger Joint,Breakfast Spot,Pizza Place,Sandwich Place,Fish & Chips Shop,Diner,Cajun / Creole Restaurant,Donut Shop
7,Avondale,Food Truck,Sandwich Place,Chinese Restaurant,Bakery,Fast Food Restaurant,Diner,Donut Shop,Korean Restaurant,Restaurant,Hot Dog Joint
9,Beverly,Pizza Place,Chinese Restaurant,Italian Restaurant,Bakery,Burger Joint,Wings Joint,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
10,Bridgeport,Chinese Restaurant,Wings Joint,Korean Restaurant,Pizza Place,Sandwich Place,Fast Food Restaurant,Donut Shop,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant
16,Clearing,Food,Pizza Place,Fried Chicken Joint,Bakery,Restaurant,Mexican Restaurant,Donut Shop,Cuban Restaurant,Czech Restaurant,Deli / Bodega
18,Dunning,American Restaurant,Deli / Bodega,Wings Joint,Falafel Restaurant,Czech Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant
19,East Garfield Park,Burger Joint,American Restaurant,Food,Diner,Wings Joint,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Donut Shop,Eastern European Restaurant
21,Edgewater,Mexican Restaurant,Sushi Restaurant,Sandwich Place,Deli / Bodega,Asian Restaurant,Restaurant,Thai Restaurant,Fast Food Restaurant,Café,Greek Restaurant


In [69]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 1, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Hermosa,Latin American Restaurant,Mexican Restaurant,Food Truck,Food,Diner,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Donut Shop,Eastern European Restaurant
32,Humboldt Park,Food Truck,Café,Latin American Restaurant,Wings Joint,Ethiopian Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop
39,Lincoln Park,Food Truck,Wings Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant


In [70]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 2, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Auburn Gresham,Fast Food Restaurant,American Restaurant,Greek Restaurant,BBQ Joint,Dim Sum Restaurant,Wings Joint,Falafel Restaurant,Deli / Bodega,Diner,Donut Shop
14,Chatham,Fast Food Restaurant,Donut Shop,Wings Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Eastern European Restaurant
17,Douglas,Fast Food Restaurant,Sandwich Place,Wings Joint,Seafood Restaurant,Asian Restaurant,Chinese Restaurant,Donut Shop,Food,Pizza Place,African Restaurant
25,Fuller Park,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Wings Joint,Eastern European Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop
69,Washington Park,Fast Food Restaurant,Fried Chicken Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
71,West Englewood,Fast Food Restaurant,American Restaurant,Wings Joint,French Restaurant,Food Truck,Food Court,Food,Fish & Chips Shop,Filipino Restaurant,Creperie
72,West Garfield Park,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Sandwich Place,Food,Taco Place,Middle Eastern Restaurant,Donut Shop,Czech Restaurant,Deli / Bodega
75,West Ridge,Fast Food Restaurant,Asian Restaurant,Donut Shop,Falafel Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Eastern European Restaurant,Ethiopian Restaurant
77,Woodlawn,Fast Food Restaurant,BBQ Joint,Falafel Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant


In [71]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 3, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Ashburn,Food,BBQ Joint,Pizza Place,Italian Restaurant,Wings Joint,Eastern European Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
13,Calumet Heights,Food,Deli / Bodega,Wings Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
20,East Side,Food,Wings Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
28,Grand Boulevard,Food,Breakfast Spot,Pizza Place,Restaurant,Food Court,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
52,North Lawndale,Food,Fried Chicken Joint,Ethiopian Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant
58,Pullman,Sandwich Place,Food,Café,Wings Joint,Eastern European Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop
61,Roseland,Chinese Restaurant,Food,Wings Joint,Ethiopian Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant


In [72]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 4, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,Mexican Restaurant,Sandwich Place,Korean Restaurant,Latin American Restaurant,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Diner,Donut Shop,Deli / Bodega
1,Archer Heights,Mexican Restaurant,Wings Joint,Seafood Restaurant,Hot Dog Joint,Italian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Restaurant,Bakery
8,Belmont Cragin,Mexican Restaurant,Restaurant,Burger Joint,American Restaurant,BBQ Joint,Bakery,Middle Eastern Restaurant,Chinese Restaurant,Ethiopian Restaurant,Dim Sum Restaurant
11,Brighton Park,Mexican Restaurant,Pizza Place,Taco Place,Sandwich Place,Café,Burger Joint,Breakfast Spot,Food,Falafel Restaurant,Fast Food Restaurant
15,Chicago Lawn,Mexican Restaurant,Pizza Place,Fast Food Restaurant,American Restaurant,Fish & Chips Shop,Wings Joint,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
26,Gage Park,Mexican Restaurant,Bakery,Pizza Place,Asian Restaurant,Sandwich Place,Chinese Restaurant,Wings Joint,Eastern European Restaurant,Deli / Bodega,Dim Sum Restaurant
27,Garfield Ridge,Mexican Restaurant,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop
30,Hegewisch,Food,Mexican Restaurant,American Restaurant,Deli / Bodega,Chinese Restaurant,Pizza Place,Wings Joint,Eastern European Restaurant,Dim Sum Restaurant,Diner
42,Lower West Side,Mexican Restaurant,Bakery,Pizza Place,Food Truck,Breakfast Spot,Donut Shop,Food,Latin American Restaurant,Café,Seafood Restaurant
43,McKinley Park,Chinese Restaurant,Diner,Mexican Restaurant,Burger Joint,Seafood Restaurant,Cantonese Restaurant,Wings Joint,Deli / Bodega,Dim Sum Restaurant,Donut Shop


In [73]:
Chicago_merged.loc[Chicago_merged['Cluster Labels'] == 5, Chicago_merged.columns[[1] + list(range(5, Chicago_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Oakland,Irish Pub,Wings Joint,Falafel Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant


In [75]:
df

,Neighborhood,Borough,Latitude,Longitude
0,"Albany Park, Mayfair, North Mayfair, Ravenswoo...",Albany Park,41.9719,-87.7162
1,Archer Heights,Archer Heights,41.8114,-87.7262
2,"Armour Square, Chinatown, Wentworth Gardens",Armour Square,41.84,-87.6331
3,"Wrightwood, Scottsdale, Crestline, Beverly Vie...",Ashburn,41.7475,-87.7112
4,"Auburn Gresham, Gresham",Auburn Gresham,41.7505,-87.6643
